## Part 1
Write your scraping (spider) code here:

[hint](https://stackoverflow.com/questions/34589064/beautifulsoup-get-the-class-text) on get text class in html tag 

In [ ]:
import pandas as pd
import json
import re

In [ ]:
from bs4 import BeautifulSoup
import requests

books = []
books_json = []

#all of page link in this web (50 page)
for i in range (1,51):
  url = f"https://books.toscrape.com/catalogue/page-{i}.html"
  r = requests.get(url)
  soup = BeautifulSoup(r.content, "html.parser")

  #all of name books in this page link
  for div in soup.find_all('div', {'class':'image_container'}):
    
    project_href = [i['href'] for i in div.find_all('a', href=True) if i['href'] != "#"]
    #loop for scrapping in 1 books with link all of book in this page
    for href in project_href:
      link = f"https://books.toscrape.com/catalogue/{href}"
      res = requests.get(link)
      s = BeautifulSoup(res.content, "html.parser")
      #find table in this page of book
      data_table = s.find('table', {'class': 'table table-striped'})
      table_val = []

      UPC = []
      ProductType = []
      Availability = []
      reviews = []
      
      #SCRAPPING DATA FROM TABLE IN HTML (upc price avai reviews)
      
      for i in data_table.find_all('tr')[0:]:
          td_tags = i.find_all('td')
          for a in td_tags:
            #check if tr = 5 (Availability) use a regular expression
            if i.find('th') in data_table.find_all('th')[5:6]:
              string_pattern = r"\d{2}|\d{1}"
              regex_pattern = re.compile(string_pattern)
              td_val = regex_pattern.findall(a.text)
            else:
              td_val = a.text 
          table_val.append(td_val)

      #save data from table to dataframe
      dataf = pd.DataFrame(data=table_val)
      #assign data from table (dataf)
      for num in dataf:
        UPC = dataf[num][0]
        ProductType = dataf[num][1]
        Availability = dataf[num][5][0]
        reviews = dataf[num][6]
      
      #assign title from tag h1
      title = s.find(class_="col-sm-6 product_main").find("h1").text
      #assign price
      price = s.find('p', {"class": "price_color"}).get_text()
      price = float(price[1:])
      #assign ratting from tag p in class star-rating
      star = s.find('p', {"class": "star-rating"})
      star = star['class'][1]
      ratting = star
      #if check description if no description assing description = NaN
      if s.find('div', {"class": "alert alert-warning"}).find_next('div').has_attr('id'):
        description = s.find('div', {"id": "product_description"}).find_next("p").get_text()
      else: description = "NaN"

    #append data to books lists
    books.append([title,price,ratting,description,UPC,ProductType,Availability,reviews])
      

In [ ]:
#string_pattern2 = r"\d{2}|\d{1}"
#number = "available is 2"
#regex_pattern2 = re.compile(string_pattern2)
#test = regex_pattern2.findall(number)
print(dataf[num][5][0])


1


SAVE lists of books to dataframe

In [ ]:
#save lists data to dataframe
booksDF = pd.DataFrame(data=books, columns=['title','price','ratting','description','UPC','Product type','available','number of reviews'])

In [ ]:
#dataframe
booksDF

,title,price,ratting,description,UPC,Product type,available,number of reviews
0,A Light in the Attic,51.77,Three,It's hard to imagine a world without A Light i...,a897fe39b1053632,Books,22,0
1,Tipping the Velvet,53.74,One,"""Erotic and absorbing...Written with starling ...",90fa61229261140a,Books,20,0
2,Soumission,50.10,One,"Dans une France assez proche de la nôtre, un h...",6957f44c3847a760,Books,20,0
3,Sharp Objects,47.82,Four,"WICKED above her hipbone, GIRL across her hear...",e00eb4fd7b871a48,Books,20,0
4,Sapiens: A Brief History of Humankind,54.23,Five,From a renowned historian comes a groundbreaki...,4165285e1663650f,Books,20,0
...,...,...,...,...,...,...,...,...
995,Alice in Wonderland (Alice's Adventures in Won...,55.53,One,NaN,cd2a2a70dd5d176d,Books,1,0
996,"Ajin: Demi-Human, Volume 1 (Ajin: Demi-Human #1)",57.06,Four,High school student Kei Nagai is struck dead i...,bfd5e1701c862ac3,Books,1,0
997,A Spy's Devotion (The Regency Spies of London #1),16.97,Five,"In England’s Regency era, manners and elegance...",19fec36a1dfb4c16,Books,1,0
998,1st to Die (Women's Murder Club #1),53.98,One,"James Patterson, bestselling author of the Ale...",f684a82adc49f011,Books,1,0


SAVE to json file (name = data_bookINDEX.json)

In [ ]:
#save dataframe to json file
booksJSON = booksDF.to_json('data_bookINDEX.json',orient="index")

In [ ]:
#load json file
fileJSON = open('/content/data_bookINDEX.json','r')
data_json = json.loads(fileJSON.read())
#check data in json file index 0
data_json['0']

{'title': 'A Light in the Attic',
 'price': 51.77,
 'ratting': 'Three',
 'description': "It's hard to imagine a world without A Light in the Attic. This now-classic collection of poetry and drawings from Shel Silverstein celebrates its 20th anniversary with this special edition. Silverstein's humorous and creative verse can amuse the dowdiest of readers. Lemon-faced adults and fidgety kids sit still and read these rhythmic words and laugh and smile and love th It's hard to imagine a world without A Light in the Attic. This now-classic collection of poetry and drawings from Shel Silverstein celebrates its 20th anniversary with this special edition. Silverstein's humorous and creative verse can amuse the dowdiest of readers. Lemon-faced adults and fidgety kids sit still and read these rhythmic words and laugh and smile and love that Silverstein. Need proof of his genius? RockabyeRockabye baby, in the treetopDon't you know a treetopIs no safe place to rock?And who put you up there,And you

save dataframe to csv (name = data_books.csv)

In [ ]:
#save dataframe to csv
booksDF.to_csv('data_books.csv')

In [ ]:
#show csv
books_details = pd.read_csv('/content/data_books.csv')
pd.set_option('expand_frame_repr', False)

books_details = books_details.drop(books_details.columns[[0]], axis = 1)
print(books_details)

                                                 title  price ratting                                        description               UPC Product type  available  number of reviews
0                                 A Light in the Attic  51.77   Three  It's hard to imagine a world without A Light i...  a897fe39b1053632        Books         22                  0
1                                   Tipping the Velvet  53.74     One  "Erotic and absorbing...Written with starling ...  90fa61229261140a        Books         20                  0
2                                           Soumission  50.10     One  Dans une France assez proche de la nôtre, un h...  6957f44c3847a760        Books         20                  0
3                                        Sharp Objects  47.82    Four  WICKED above her hipbone, GIRL across her hear...  e00eb4fd7b871a48        Books         20                  0
4                Sapiens: A Brief History of Humankind  54.23    Five  From a renowned his

## Part 2 
Please perform data manipulation on the crawled dataset and answer the below questions. 

Q1. **[1000]** What is the total number of books on the website?

Q2. **[35.07035]**  What is the average book price?

Q3. **[8.585]**  what is the average Availability of books?

Q4. **[rating distribution one : 226
      rating distribution two : 196
      rating distribution three : 203
      rating distribution four : 179
      rating distribution five : 196]** What is the distribution of book ratings? (count the number of a book for each star rate) 


Note that you must show with the code below how you reached the answer.


In [ ]:
#Q1 What is the total number of books on the website?
sum_ofbooks = len(books_details.index)
print(sum_ofbooks)
#Q2  What is the average book price?
avg_ofprice = books_details['price'].mean()
print(avg_ofprice)
#Q3 what is the average Availability of books?
avg_ofbook = books_details['available'].mean()
print(avg_ofbook)
#Q4 What is the distribution of book ratings?
book_ratting_one = books_details['ratting'].value_counts()['One']
book_ratting_two = books_details['ratting'].value_counts()['Two']
book_ratting_three = books_details['ratting'].value_counts()['Three']
book_ratting_four = books_details['ratting'].value_counts()['Four']
book_ratting_five = books_details['ratting'].value_counts()['Five']
print('rating distribution one : '+str(book_ratting_one))
print('rating distribution two : '+str(book_ratting_two))
print('rating distribution three : '+str(book_ratting_three))
print('rating distribution four : '+str(book_ratting_four))
print('rating distribution five : '+str(book_ratting_five))

1000
35.07035
8.585
rating distribution one : 226
rating distribution two : 196
rating distribution three : 203
rating distribution four : 179
rating distribution five : 196


                                                 title  price rating                                        description               UPC Product Type  Availability  Number of reviews
0                                 A Light in the Attic  51.77  Three  It's hard to imagine a world without A Light i...  a897fe39b1053632        Books            22                  0
1                                   Tipping the Velvet  53.74    One  "Erotic and absorbing...Written with starling ...  90fa61229261140a        Books            20                  0
2                                           Soumission  50.10    One  Dans une France assez proche de la nôtre, un h...  6957f44c3847a760        Books            20                  0
3                                        Sharp Objects  47.82   Four  WICKED above her hipbone, GIRL across her hear...  e00eb4fd7b871a48        Books            20                  0
4                Sapiens: A Brief History of Humankind  54.23   Five  From a ren